# Ontbrekende Waardes

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Laad de data
data_bestand = '../data/data.csv'
data = pd.read_csv(data_bestand) 

# Huizen zonder prijs mogen er al uit.
data = data[data['Price'].notna()]

# Selecteer doel
y = data.Price

# Om het simpel te houden, gebruiken we alleen numerieke voorspellers
data_predictors = data.drop(['Price'], axis=1)
X = data_predictors.select_dtypes(exclude=['object'])

# Verdeel data in trainings- en validatiesubsets
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [ ]:
# Functie voor het vergelijken van verschillende benaderingen
def score_dataset(X_train, X_test, y_train, y_test):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    voorspel = model.predict(X_test)
    return mean_absolute_error(y_test, voorspel)

In [ ]:
# Functie voor het vergelijken van verschillende benaderingen
def score_dataset(X_train, X_test, y_train, y_test):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    voorspel = model.predict(X_test)
    return mean_absolute_error(y_test, voorspel)

# Benadering 1 (Kolommen met ontbrekende waarden verwijderen)

In [ ]:
# Namen van kolommen met ontbrekende waarden ophalen
kolommen_met_onbrekende_waarden = [kol for kol in X_train.columns if X_train[kol].isnull().any()]

# Kolommen verwijderen in trainings- en validatiegegevens (controlegegevens)
beperkt_X_train = X_train.drop(kolommen_met_onbrekende_waarden, axis=1)
beperkt_X_test = X_test.drop(kolommen_met_onbrekende_waarden, axis=1)

print("MAE van Benadering 1 (kolommen met ontbrekende waarden verwijderen):")
print(score_dataset(beperkt_X_train, beperkt_X_test, y_train, y_test))

# benadering 2 (imputatie)

In [ ]:
from sklearn.impute import SimpleImputer

# Imputatie
mijn_imputatie = SimpleImputer()
imputatie_X_train = pd.DataFrame(mijn_imputatie.fit_transform(X_train))
imputatie_X_test = pd.DataFrame(mijn_imputatie.transform(X_test))

# Imputatie verwijderde kolomnamen; plaatste ze terug
imputatie_X_train.columns = X_train.columns
imputatie_X_test.columns = X_test.columns

print("MAE van benadering 2 (Imputatie):")
print(score_dataset(imputatie_X_train, imputatie_X_test, y_train, y_test))

In [ ]:
# Kopie maken om te voorkomen dat originele gegevens worden gewijzigd (bij imputatie)
X_train_plus = X_train.copy()
X_test_plus = X_test.copy()


# Nieuwe kolommen maken die aangeven wat er wordt geïmputeerd
for col in kolommen_met_onbrekende_waarden:
   X_train_plus[col + '_ontbrak'] = X_train_plus[col].isnull()
   X_test_plus[col + '_ontbrak'] = X_test_plus[col].isnull()


# Imputatie
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_test_plus))


# Imputatie verwijderde kolomnamen; zet ze terug
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_test_plus.columns


print("MAE van benadering 3 (Een uitbreiding op Imputatie):")
print(score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_test))

In [ ]:
# Vorm van trainingsgegevens (num_rows, num_columns)
print(X_train.shape)

# Aantal ontbrekende waarden in elke kolom van trainingsgegevens

ontbrekende_waarden_per_kolom = (X_train.isnull().sum())
print(ontbrekende_waarden_per_kolom[ontbrekende_waarden_per_kolom > 0])